In [1]:
import numpy as np
import shorttext

//anaconda/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
# loading the whole ESV Bible
import biblebooks.bibledocs_iterator as bit
dbconn = bit.get_sqlite3_dbconn("ESVCrossway.sqlite")
biblecorpus = bit.generate_classdict_chapters(dbconn)

In [4]:
from biblebooks import BibleAbbrDict

In [6]:
# loading FastText model
wvmodel = shorttext.utils.load_fasttext_model('/Users/hok/Data/FastText/wiki.simple/wiki.simple.bin')

In [7]:
# define neural network

In [8]:
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, LSTM, Activation, Input
from keras.models import Sequential, Model
from keras.regularizers import l2

In [41]:
maxlen = 20

input_x = Input(shape=(maxlen, wvmodel.vector_size))
output_y = LSTM(1024, return_sequences=True)(input_x)
print(output_y.shape)
output_y = Conv1D(filters=1200, kernel_size=2)(output_y)
print(output_y.shape)
output_y = MaxPooling1D(pool_size=19)(output_y)
print(output_y.shape)
output_y = Flatten()(output_y)
print(output_y.shape)
output_y = Dense(2, activation='sigmoid')(output_y)
print(output_y.shape)

kmodel = Model(inputs=input_x, outputs=output_y)

(?, ?, 1024)
(?, ?, 1200)
(?, ?, 1200)
(?, ?)
(?, 2)


In [42]:
kmodel.compile(loss='categorical_crossentropy', optimizer='adam')

In [38]:
maxlen = 20

smodel = Sequential()
smodel.add(LSTM(1024, return_sequences=True, input_shape=(maxlen, wvmodel.vector_size)))
smodel.add(Conv1D(filters=1200, kernel_size=2))
smodel.add(MaxPooling1D(pool_size=maxlen-2+1))
smodel.add(Flatten())
smodel.add(Dense(2, activation='sigmoid'))

smodel.compile(loss='categorical_crossentropy', optimizer='adam')

In [19]:
newdict = {}
newdict['ot'] = [sent for otbookabbr in BibleAbbrDict.otbookdict 
                 for chap in range(1, BibleAbbrDict.numchaps[otbookabbr]+1) 
                 for sent in biblecorpus[otbookabbr+'_'+str(chap)]]
newdict['nt'] = [sent for otbookabbr in BibleAbbrDict.ntbookdict 
                 for chap in range(1, BibleAbbrDict.numchaps[otbookabbr]+1) 
                 for sent in biblecorpus[otbookabbr+'_'+str(chap)]]

In [32]:
classifier = shorttext.classifiers.VarNNEmbeddedVecClassifier(wvmodel, maxlen=maxlen)

In [43]:
classifier.train(newdict, kmodel)

Epoch 1/10
 2784/35885 [=>............................] - ETA: 11:06 - loss: 0.5793

KeyboardInterrupt: 

In [34]:
classlabels, train_embedvec, indices = classifier.convert_trainingdata_matrix(newdict)

In [35]:
indices

array([[1, 0],
       [1, 0],
       [1, 0],
       ...,
       [0, 1],
       [0, 1],
       [0, 1]])